# **Проверка конфлюэнтности по перекрытию**

## **Парсер**

In [2]:
import re

rule_format = r'[ \t]*([a-z]+)[ \t]*->[ \t]*([a-z]*)[ \t]*'

def parse(text):
    srs = []
    for line in text.strip().split('\n'):
        if line == '': continue
        if not re.fullmatch(rule_format, line):
            raise Exception(f'The rule \"{line}\" has incorrect format')
        srs.append(list(map(lambda x: x.strip(), line.split('->'))))
    return srs

## **Проверка на перекрытие**

In [3]:
def has_overlap(term, other_term=None):
    if other_term == None:
        for i in range(1, len(term) // 2 + 1):
            if term[:i] == term[-i:]:
                return True
        return False
    for i in range(min(len(term), len(other_term)) + 1):
        if term[i:] == other_term[-i:] or term[-i:] == other_term[i:]:
            return True
    return False

## **Проверка конфлюэнтности**

In [4]:
def confluence_verdict(srs):
    for i in range(len(srs)):
        if has_overlap(srs[i][0]):
            return ('The system may not be confluence', f'Overlapping term: {srs[i][0]}')
        for j in range(i + 1, len(srs)):
            if has_overlap(srs[i][0], srs[j][0]):
                return ('The system may not be confluence', f'Overlapping terms: {srs[i][0]}, {srs[j][0]}')
    return ('The system is confluence', None)

## **Тесты**

In [6]:
from IPython.core.display import display, HTML

tests_count = 5

for i in range(tests_count):
    with open(f'tests/test_{i}.txt') as f:
        try:
            srs = parse(f.read())
            verdict, overlap = confluence_verdict(srs)
            display(HTML(
                f"""
                <h3>Test {i + 1}</h3>
                <div>
                    <b>{verdict}</b><br>
                    <i>{overlap if overlap != None else ''}</i>
                </div>
                """
            ))
        except Exception as err:
            display(HTML(
                f"""
                <h3>Test {i + 1}</h3>
                <div><b><font color='red'>Parse error: </font></b>{err}</div>
                """
            ))